# Graphical User Interface for Single Tello Operation 
## -Machine Learning Based Automated Target Detection and Tracking
## -Live Video Downlink and Telemetry Monitoring 
## -Manual Flight Control



### Imports and Globals

In [18]:
# Standard library imports
import threading 
import socket
import queue
from time import sleep

# Third party imports
import numpy as np
import cv2
from IPython.display import display
import ipywidgets.widgets as widgets

# Local application imports
from tello import Tello
from stream_camera import StreamCamera
from ml_process import MLProcess

# Globals
TelloIP = '192.168.10.1'
TelloVideoPort = 11111   # h.264 video stream from the Tello
TelloMovementStepSize = 25 # distance in CM for movement changes

### Define The Graphical User Interface Elements
The user intreface is built using Jupyter Widgets...

Explain event driven programming using on submit and observe methods...

In [19]:
# The status data area is created using a grid widget. This widget allows all of the elements to be placed at uniformly
# spaced intervals. A list of Text widgets is created (status_items) to hold each piece of status data from the Tello.
# One widget from the list is then assigned a specific spot on the grid to create the data layout that is desired.
# The status thread will directly update the values for each widget in the background. The order of the data in the list
# is just the prder defined the the SDK. For example status_item[15] is the battery voltage.

status_grid = widgets.GridspecLayout (9,4)

sg_style = {'description_width':'40px'}

# Create a text box for each of the status items sent from the Tello
status_items = [ widgets.Text(value = str(index),
                              description=str(index),
                              layout=widgets.Layout(width='100px'),
                              style=sg_style, 
                              disabled=True) for index in range(21) ]

#status_grid[0,0] = widgets.Label(value="Found MP:")
status_grid[0,0] = status_items[0]
                          
for i in [1,2,3]:
    status_grid[0,i] = status_items[i]

status_grid[2,0] = widgets.Label(value="Attitude:")
for i in [1,2,3]:
    status_grid[2,i] = status_items[i+4]
    
status_grid[3,0] = widgets.Label(value="Velocity:")
for i in [1,2,3]:
    status_grid[3,i] = status_items[i+7]

status_grid[4,0] = widgets.Label(value="Acc:")
for i in [1,2,3]:
    status_grid[4,i] = status_items[i+17]

status_grid[5,0] = widgets.Label(value="Temp:")
status_grid[5,1] = status_items[11]
status_grid[5,2] = status_items[12]
    
status_grid[6,0] = widgets.Label(value="Time:")
status_grid[6,1] = status_items[13]
status_grid[6,2] = status_items[17]

status_grid[7,0] = widgets.Label(value="Altitdue:")
status_grid[7,1] = status_items[14]
status_grid[7,2] = status_items[16] 
    
status_grid[8,0] = widgets.Label(value="Battery:")
status_grid[8,1] = status_items[15]


In [20]:
# create system status indicators
ss_style = {'description_width':'110px'}
ss_title_bar = widgets.Text(value = 'x', description = 'IP Address:')
ss_communication = widgets.Valid(value=False, description='Communication', style=ss_style)
ss_status_stream = widgets.Valid(value=False, description='Status Stream', style=ss_style)
#ss_command_mode = widgets.Valid(value=False, description='Command State', style=ss_style)
ss_video_stream = widgets.Valid(value=False, description='Video Stream', style=ss_style)
ss_ml_process = widgets.Valid(value=False, description='ML Process', style=ss_style)

# create a composite control
system_status= widgets.VBox([ss_title_bar,
                             ss_communication,
                             ss_status_stream,
                             #ss_command_mode,
                             ss_video_stream,
                             ss_ml_process], 
                            layout=widgets.Layout(align_self='auto'))

In [21]:
# create flight control buttons
fc_button_layout = widgets.Layout(width='80px', height='60px', align_self='center')
# TODO - button state should init to current global var
fc_center_button = widgets.Button(description='Takeoff', button_style='success', layout=fc_button_layout)
fc_forward_button = widgets.Button(description='Forward', layout=fc_button_layout)
fc_backward_button = widgets.Button(description='Backward', layout=fc_button_layout)
fc_left_button = widgets.Button(description='Rotate Left', layout=fc_button_layout)
fc_right_button = widgets.Button(description='Rotate Right', layout=fc_button_layout)
fc_middle_box = widgets.HBox([fc_left_button, fc_center_button, fc_right_button], layout=widgets.Layout(align_self='center'))
fc_button_box = widgets.VBox([fc_forward_button, fc_middle_box, fc_backward_button])

fc_altitude_slider = widgets.IntSlider(
                    value=0.5,
                    min=20,
                    max=200,
                    step=5,
                    description='Alt:',
                    disabled=False,
                    continuous_update=True,
                    orientation='vertical',
                    readout=True,
                    readout_format='1d'
                )

# create a composite control
flight_controls = widgets.HBox([fc_button_box, fc_altitude_slider], layout=widgets.Layout(align_self='center'))

In [22]:
# create a live video window
# Tello's native video resolution is 720p (1280 x 720)
video_frame = widgets.Image(format='jpeg',
                            width=300, 
                            height=300, 
                            description='Live Video Feed',
                            layout=widgets.Layout(align_self='center')
                           ) # 170,90
#video_frame = video_frame.from_file('test_image.jpg')

In [23]:
# create target controls
target_selection = widgets.Dropdown(
                        options=[('None', -1), ('Person', 1), ('Cup', 47), ('Cat', 17)],
                        value=-1,
                        description='Target:',
                        )

# TODO - button state should init to current global var
detect_button = widgets.Button(description='Detect', layout=widgets.Layout(align_self='center'))
track_button = widgets.Button(description='Track', layout=widgets.Layout(align_self='center'))
target_activation = widgets.HBox([detect_button, track_button])
detection_confidence = widgets.Text(description = 'Confidence:',
                                 value = '')

detection_progress = widgets.FloatProgress(
                    value=0.0,
                    min=0.0,
                    max=1.0,
                    step=1,
                    description='Accuracy:',
                    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
                    orientation='horizontal'
                )

target_controls = widgets.VBox([target_selection,  
                                detection_confidence,
                                target_activation] )

In [24]:
#create machine learning controls
epochs_widget = widgets.IntText(description='epochs', value=1, layout=widgets.Layout(align_self='center'))
eval_button = widgets.Button(description='evaluate', layout=widgets.Layout(align_self='center'))
train_button = widgets.Button(description='train')
loss_widget = widgets.FloatText(description='loss')
accuracy_widget = widgets.FloatText(description='accuracy')
progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='progress')

machine_learning_controls = widgets.VBox([ epochs_widget,
                                           eval_button,
                                           loss_widget,
                                           accuracy_widget,
                                           progress_widget ])

In [25]:
# create command area controls
command_line = widgets.Text(description='Commands:',
                            value='',
                            placeholder='command takeoff land flip forward back left right up down cw ccw speed speed?',
                            layout = widgets.Layout(width='800px'))
# response line
response_line = widgets.Textarea(description='Response:',
                                 value='',
                                 layout = widgets.Layout(width='800px'))

command_area = widgets.VBox([ command_line,response_line ])

### Define and Register Callback Handlers

In [26]:
# define and register command line text box callback
def on_command_entered(widget_triggered):    
    msg = widget_triggered.value
    tello.send_command(msg)

command_line.on_submit(on_command_entered)

def target_selection_change(change):
    mlp.target_selection = change.new

# define and register flight control button callbacks
def on_center_button_clicked(button):
    if tello.is_flying == False:
        tello.takeoff()
        tello.is_flying = True
        #fc_altitude_slider.value = tello.get_height()
        print(tello.get_height())
        button.description='Land'
        button.button_style='danger'
    else:
        tello.land()
        tello.is_flying = False
        button.description='Takeoff'
        button.button_style='success'
        
def on_forward_button_clicked(button):
    tello.move_forward(TelloMovementStepSize)
    
def on_backward_button_clicked(button):
    tello.move_backward(TelloMovementStepSize)

def on_left_button_clicked(button):
    tello.rotate_ccw(30)

def on_right_button_clicked(button):
    tello.rotate_cw(30)

def fc_altitude_slider_changed(change):
    if change.new > change.old:
        tello.move_up(change.new)
    else:
        tello.move_down(change.new)

        
# define  control button callbacks   
def on_detect_button_clicked(button):
    
    if mlp.detections_active == False:
        mlp.detections_active = True
        button.button_style='success'
    else:
        mlp.detections_active = False
        button.button_style='danger'
    
def on_track_button_clicked(button):
    
    if mlp.tracking_active == False:
        mlp.tracking_active = True
        button.button_style='success'
    else:
        mlp.tracking_active = False
        button.button_style='danger'
    

# register flight control button callbacks
fc_center_button.on_click(on_center_button_clicked)
fc_forward_button.on_click(on_forward_button_clicked)
fc_backward_button.on_click(on_backward_button_clicked)
fc_left_button.on_click(on_left_button_clicked)
fc_right_button.on_click(on_right_button_clicked)
fc_altitude_slider.observe(fc_altitude_slider_changed, names='value')

#define target control buttons
detect_button.on_click(on_detect_button_clicked)
track_button.on_click(on_track_button_clicked)
target_selection.observe(target_selection_change, names='value')

### Create and Display the Composite GUI

The AppLayout widget is used to organize the individual elements of the GUI. The widget is divided into five 
main areas - a header, a footer and three middle areas - left, center and right. Each of the sections defined above
are assigned to the AppLayout widget and then the composit GUI is displayed.

In [27]:
left = widgets.VBox([status_grid, 
                     system_status], 
                    layout=widgets.Layout(align_self='auto'))

center = widgets.VBox([video_frame, flight_controls])
right = widgets.VBox([target_controls])

composit_gui = widgets.AppLayout(header = None,
                                 left_sidebar = left,
                                 center = center,
                                 right_sidebar = right,
                                 pane_widths = [2,2,2],
                                 pane_heights = ['50px',3,1],
                                 footer = command_area)
display(composit_gui)

AppLayout(children=(VBox(children=(Text(value='', description='Commands:', layout=Layout(width='800px'), place…

### Create a Tello object for aircraft Command, Control and Communication

In [11]:
def communication_started_change(change):
    ss_communication.value = change.new
    
def status_started_change(change):
    ss_status_stream.value = change.new

def status_data_change(change):
    data = change.new
    data = data[:-5].split(';') # convert the data into an array of individual strings, cutting off the trailing ';\r\n'
    
    global status_items
    
    # The elements defined in the GUI rely on the data being saved to the list in order as defined in the SDK
    # todo - test to make sure data lenght = list size
    for index in range (len(data)):
        status_items[index].description = data[index].split(':')[0]
        status_items[index].value = data[index].split(':')[1]              

def response_change(change):
    response_line.value = change.new

    
# Create a Tello and start the Command Mode Heartbeat and Status threads
try:
    tello = Tello.instance(tello_ip = '192.168.10.1',
                           local_ip='192.168.10.2',
                           command_timeout=3.0
                           )   
except (RuntimeError, OSError) as msg:
    tello = None
    response_line.value = repr(msg)
else:
    tello.unobserve_all()
    tello.observe(communication_started_change, names='communication_started')
    tello.observe(response_change, names='response')
    tello.start_communication()
    tello.send_command('command')
    tello.observe(status_started_change, names='status_started')
    tello.observe(status_data_change, names='status_data')
    tello.start_status()

>> send cmd: command
b'\xcc\x18\x01\xb9\x88V\x00\xb0\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00I\x00\x00\xa8\x0f\x00\x06\x00\x00\x00\x00\x00k\xcc'


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tom/Projects/kipr-areial-dev/KT-GUI-AI-repo/tello.py", line 91, in _receive_cmd_replies
    self._error_response = response.decode(encoding='utf-8') # converts a byte array to a string
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte



### Create a camera object to receive video from the Tello
The Tello uses WiFi for all of its communication.

The video socket is created by the OpenCV VideoCapture method defined in the TelloCamera class.
*** Important Note: the current Tello firmware can only stream video if the Tello is directly connected to the host 
computer - i.e. at 192.168.10.1 without using a router. If you assign it to an existing netowork it will not stream video.
The streamon command will not return an error.

In [12]:
def stream_started_change(change):
    ss_video_stream.value = change.new
    
# Create a streaming camera object for the Tello camera start the video stream

tello.send_command('streamon')

try:
    camera = StreamCamera.instance() # TODO Need to add ip address parameter   
except (cv2.error,RuntimeError) as msg:
    camera = None
    response_line.value = repr(msg)
else:
    camera.unobserve_all()
    camera.observe(stream_started_change, names='started')
    camera.start()


>> send cmd: streamon


### Create a Machine Learning Process (SDD-DNN) for Image Processing

In [13]:
def image_change(change):
    video_frame.value = bytes(cv2.imencode('.jpg', change['new'])[1])
    
    # if tracking activated and something found, display the confidentce
    if mlp.detections_active and bool(mlp.filtered_detections):
        detection_confidence.value = str(mlp.filtered_detections[0]['confidence'])
    else:
        detection_confidence.value = 'No Detections'

def mlp_started_change(change):
    ss_ml_process.value = change.new
    
# Create a ML process and start the thread
try:
    mlp = MLProcess.instance(camera=camera, tello=tello)   
except (RuntimeError) as msg:
    mlp = None
    response_line.value = repr(msg)
else:
    mlp.unobserve_all()
    mlp.observe(mlp_started_change, names='started')
    mlp.observe(image_change, names='processed_image')
    mlp.start()


In [14]:
temp = mlp.filtered_detections

In [15]:
bool(mlp.filtered_detections)

False

In [16]:
temp

[]

In [17]:
accuracy_widget.value = temp[0]['confidence']

IndexError: list index out of range

In [ ]:
detection_accuracy.value = str(10)